<a href="https://colab.research.google.com/github/sammanfatima/Pytorch-Journey/blob/main/02_Pytorch_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

1. torch core pytorch library used for creating tensors(multi dim arrays) for storing large amount of data, initializing, math operations CPU/GPU support

2. torch.nn helps you to create neural network easily, define models, create layers

3. torch.optim to update weights during training

4. torch.nn.functional(functionl API) for activation functions(relu, softmax) and other functions.

5. from torch.utils.data to feed data into batches and shuffle it during training

6. torchvision.datasets ready made datasets for images

7. torchvision.transforms change image preparing your images so the network can understand them better
like resize image, normalize img

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# **Create Fully Connected Network**

* nn.Linear means fully connected newtork(dense)
* input_size number of features ,50 no neurons (hidden unit)
* num_classes no. of output neurons

# **Forward Propagation**

* fc1 → hidden layer

* ReLU → activation for hidden layer

* fc2 → output layer, produces raw scores (logits)




In [ ]:
class NN(nn.Module): # inherit from pytorch's nn.module provide all tools to create layers, store weights
  def __init__(self, input_size, num_classes): # constructor it automatically called when you create an instance of class
    super(NN, self).__init__() # call the constructor of parent class
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
model = NN(784, 10)
x = torch.rand(64, 784)
print(model(x).shape)



torch.Size([64, 10])


# **Set Device**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **Hyperparameters**

In [ ]:
input_size = 784 # no of input features
num_classes = 10 # no of output classes
learning_rate = 0.01 # Controls how fast the model learns
batch_size = 64 # Number of samples processed at one time
num_epochs = 1 # Number of times the model sees the entire dataset

# **Load Data**

In [ ]:
# train dataset
train_dataset = datasets.MNIST(root= '/dataset', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle=True)

# test dataset
test_dataset = datasets.MNIST(root= '/dataset', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= batch_size, shuffle=True)


# **Initialize Network**

In [ ]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# **Loss and Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss() #criterion is loss function which measures how wrong the output is
# cross entropy used for classification problems
optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer update weights so the loss become smaller
# Adam is a smart optimizer that learns how to update weights efficiently

# **Train Network**

In [ ]:
for epoch in range(num_epochs):
  for batch_idx,(data, target) in enumerate(train_loader):# take small batches and then process see model batch by batch

  # get data to cuda if possible
    data = data.to(device=device) # batch input features
    target = target.to(device=device) # batch correct labels

    # get to correct shape
    data = data.reshape(data.shape[0], -1)

    #print(data.shape) # size and structure of the batch

    # forward
    scores = model(data)
    loss = criterion(scores, target)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or Adam step
    optimizer.step()

# **Check accuracy on train and test**

In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      x = x.reshape(x.shape[0], -1)
      scores = model(x)

      scores = model(x)
      _,predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}')

  model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)









Checking accuracy on training data
Got 57505/ 60000 with accuracy 95.84166666666667
Checking accuracy on test data
Got 9532/ 10000 with accuracy 95.32000000000001
